In [2]:
from pyspark.sql import SparkSession
spark=SparkSession.builder.appName("Ventas Mundiales").getOrCreate()

In [3]:
from pyspark.sql.types import *
estructura=StructType(
    [
        StructField("ProductID",IntegerType()),
        StructField("Date",DateType()),#indicar el formato con dateFormat
        StructField("Zip",StringType()),
        StructField("Units",IntegerType()),
        StructField("Revenue",DecimalType(10,2)),
        StructField("Country",StringType())
    ]
)

ventas=spark.read.csv("hdfs://localhost:9000/ventas/Ventas1",header=True,
                     schema=estructura,dateFormat="mm/dd/yyyy")

In [4]:
ventas.printSchema(),ventas.head(2)

root
 |-- ProductID: integer (nullable = true)
 |-- Date: date (nullable = true)
 |-- Zip: string (nullable = true)
 |-- Units: integer (nullable = true)
 |-- Revenue: decimal(10,2) (nullable = true)
 |-- Country: string (nullable = true)



(None,
 [Row(ProductID=726, Date=datetime.date(1999, 1, 15), Zip='75056 CEDEX 01', Units=1, Revenue=Decimal('115.45'), Country='France'),
  Row(ProductID=1909, Date=datetime.date(1999, 1, 15), Zip='75056 CEDEX 01', Units=2, Revenue=Decimal('398.90'), Country='France')])

In [5]:
ventas.count()

390068

In [6]:
ventas.describe().show()

+-------+------------------+------------------+------------------+-----------------+-------+
|summary|         ProductID|               Zip|             Units|          Revenue|Country|
+-------+------------------+------------------+------------------+-----------------+-------+
|  count|            390068|            390068|            390068|           390068| 390068|
|   mean|1228.5664217521048| 40498.70806110035|1.0391495841750669|       468.142721|   null|
| stddev| 699.8131279250586|34281.270141426176|0.3939632542672234|413.5918991814394|   null|
|    min|                 1|     06001 CEDEX 1|                 1|             0.00| Canada|
|    max|              2412|               V7Y|                77|         43194.11| France|
+-------+------------------+------------------+------------------+-----------------+-------+



In [7]:
ventas.where("ProductID=726").describe().show()

+-------+---------+-----------------+-------------------+-----------------+-------+
|summary|ProductID|              Zip|              Units|          Revenue|Country|
+-------+---------+-----------------+-------------------+-----------------+-------+
|  count|     2700|             2700|               2700|             2700|   2700|
|   mean|    726.0|41539.22784810127| 1.0585185185185184|       122.895230|   null|
| stddev|      0.0|34442.25975499514|0.25004932753379183|29.36378162539805|   null|
|    min|      726|    06003 CEDEX 1|                  1|           110.20| Canada|
|    max|      726|              V7W|                  3|           362.09| France|
+-------+---------+-----------------+-------------------+-----------------+-------+



In [8]:
ventas.where("ProductID=726").agg({"Units":"sum"}).show()

+----------+
|sum(Units)|
+----------+
|      2858|
+----------+



In [9]:
from pyspark.sql.functions import sum,translate,regexp_replace,lit
ventas.where("ProductID=726").agg(sum("Units")).show()

+----------+
|sum(Units)|
+----------+
|      2858|
+----------+



In [10]:
ventas.groupBy("ProductID").sum("Units")\
    .orderBy("sum(Units)",ascending=False).show()

+---------+----------+
|ProductID|sum(Units)|
+---------+----------+
|      475|      4730|
|      604|      4591|
|     2332|      3873|
|      792|      3732|
|      791|      3732|
|      487|      3656|
|     2045|      3654|
|      826|      3532|
|     2255|      3513|
|     2254|      3513|
|      438|      3364|
|      396|      3346|
|      785|      2901|
|      786|      2901|
|      725|      2858|
|      726|      2858|
|     1009|      2772|
|     1182|      2759|
|      609|      2753|
|      758|      2676|
+---------+----------+
only showing top 20 rows



In [11]:
ventas.groupBy("ProductID").sum("Units")\
    .orderBy("ProductID").show()

+---------+----------+
|ProductID|sum(Units)|
+---------+----------+
|        1|        13|
|        3|       256|
|        4|        37|
|        7|        10|
|        8|       199|
|        9|       103|
|       10|         6|
|       12|         7|
|       13|        21|
|       14|         3|
|       15|         2|
|       16|         7|
|       17|        79|
|       19|         8|
|       21|        17|
|       23|        12|
|       25|        12|
|       26|       417|
|       27|         6|
|       28|         2|
+---------+----------+
only showing top 20 rows



In [12]:
ventasUSA=spark.read.csv("hdfs://localhost:9000/ventas/Ventas2",sep=";",inferSchema=True)

In [13]:
ventasUSA.printSchema()

root
 |-- _c0: integer (nullable = true)
 |-- _c1: timestamp (nullable = true)
 |-- _c2: integer (nullable = true)
 |-- _c3: integer (nullable = true)
 |-- _c4: string (nullable = true)



In [14]:
ventas.columns

['ProductID', 'Date', 'Zip', 'Units', 'Revenue', 'Country']

In [15]:
ventasUSA.show(2)

+---+-------------------+-----+---+--------+
|_c0|                _c1|  _c2|_c3|     _c4|
+---+-------------------+-----+---+--------+
|833|1999-01-15 00:00:00|76108|  1|797,9475|
|837|1999-01-15 00:00:00|33436|  1|813,6975|
+---+-------------------+-----+---+--------+
only showing top 2 rows



In [16]:
ventasUSA2=ventasUSA.select(ventasUSA._c0.alias("ProductID"),
                ventasUSA._c1.cast("date").alias("Date"),
                ventasUSA._c2.cast("string").alias("Zip"),
                ventasUSA._c3.alias("Units"),
                translate(ventasUSA._c4,",",".").cast("Decimal(12,4)").alias("Revenue"),
                #regexp_replace(ventasUSA._c4,"\,",".").cast("float").alias("Revenue")           
                ).withColumn("Country",lit("USA"))

ventasUSA2.show(),ventasUSA2.printSchema()

+---------+----------+-----+-----+--------+-------+
|ProductID|      Date|  Zip|Units| Revenue|Country|
+---------+----------+-----+-----+--------+-------+
|      833|1999-01-15|76108|    1|797.9475|    USA|
|      837|1999-01-15|33436|    1|813.6975|    USA|
|      837|1999-01-15|76531|    1|813.6975|    USA|
|      838|1999-01-15|29526|    1|934.4475|    USA|
|      838|1999-01-15|33584|    1|944.9475|    USA|
|      838|1999-01-15|33947|    1|944.9475|    USA|
|      838|1999-01-15|74132|    1|929.1975|    USA|
|      838|1999-01-15|75080|    1|944.9475|    USA|
|      838|1999-01-15|85225|    1|934.4475|    USA|
|      838|1999-01-15|92647|    1|929.1975|    USA|
|      838|1999-01-15|92833|    1|934.4475|    USA|
|      843|1999-01-15|91730|    1|551.1975|    USA|
|      847|1999-01-15|17701|    1|603.6975|    USA|
|      847|1999-01-15|71446|    1|577.4475|    USA|
|      847|1999-01-15|99205|    1|603.6975|    USA|
|      852|1999-01-15|36054|    1|745.4475|    USA|
|      852|1

(None, None)

In [17]:
ventasUSA.printSchema()

root
 |-- _c0: integer (nullable = true)
 |-- _c1: timestamp (nullable = true)
 |-- _c2: integer (nullable = true)
 |-- _c3: integer (nullable = true)
 |-- _c4: string (nullable = true)



In [18]:
productos=spark.read.csv("../bi_product.txt",sep=";",inferSchema=True)

In [19]:
productos.printSchema(),productos.show()

root
 |-- _c0: integer (nullable = true)
 |-- _c1: string (nullable = true)
 |-- _c2: string (nullable = true)
 |-- _c3: string (nullable = true)
 |-- _c4: integer (nullable = true)
 |-- _c5: string (nullable = true)

+---+-----------+---+----------+---+-----+
|_c0|        _c1|_c2|       _c3|_c4|  _c5|
+---+-----------+---+----------+---+-----+
|  1|Abbas MA-01|Mix|All Season|  1|Abbas|
|  2|Abbas MA-02|Mix|All Season|  1|Abbas|
|  3|Abbas MA-03|Mix|All Season|  1|Abbas|
|  4|Abbas MA-04|Mix|All Season|  1|Abbas|
|  5|Abbas MA-05|Mix|All Season|  1|Abbas|
|  6|Abbas MA-06|Mix|All Season|  1|Abbas|
|  7|Abbas MA-07|Mix|All Season|  1|Abbas|
|  8|Abbas MA-08|Mix|All Season|  1|Abbas|
|  9|Abbas MA-09|Mix|All Season|  1|Abbas|
| 10|Abbas MA-10|Mix|All Season|  1|Abbas|
| 11|Abbas MA-11|Mix|All Season|  1|Abbas|
| 12|Abbas MA-12|Mix|All Season|  1|Abbas|
| 13|Abbas MA-13|Mix|All Season|  1|Abbas|
| 14|Abbas MA-14|Mix|All Season|  1|Abbas|
| 15|Abbas MA-15|Mix|All Season|  1|Abbas|
| 16|Abb

(None, None)

In [20]:
ventasTotales=ventas.union(ventasUSA2)
ventasTotales.cache()
ventasTotales.count()

1390068

In [21]:
ventasTotales.show()

+---------+----------+--------------+-----+---------+-------+
|ProductID|      Date|           Zip|Units|  Revenue|Country|
+---------+----------+--------------+-----+---------+-------+
|      726|1999-01-15|75056 CEDEX 01|    1| 115.4500| France|
|     1909|1999-01-15|75056 CEDEX 01|    2| 398.9000| France|
|     1961|1999-01-15|75056 CEDEX 01|    1|  97.0700| France|
|     1517|1999-01-15|75056 CEDEX 01|    1| 141.6500| France|
|      606|1999-01-15|75056 CEDEX 01|    1| 314.7400| France|
|     1518|1999-01-15|75056 CEDEX 01|    1| 141.6500| France|
|      786|2002-01-31|         75001|    1|  68.2000| France|
|      727|1999-01-15|75063 CEDEX 02|    2| 125.9000| France|
|      559|1999-01-15|75063 CEDEX 02|    1| 585.6400| France|
|      728|1999-01-15|75063 CEDEX 02|    2| 125.9000| France|
|      964|2003-01-10|75063 CEDEX 02|    1| 320.2000| France|
|      475|2003-01-11|75063 CEDEX 02|    1|1037.4000| France|
|     2055|2002-01-05|75063 CEDEX 02|    1| 569.5700| France|
|     15

In [22]:
ventasTotales.printSchema()

root
 |-- ProductID: integer (nullable = true)
 |-- Date: date (nullable = true)
 |-- Zip: string (nullable = true)
 |-- Units: integer (nullable = true)
 |-- Revenue: decimal(12,4) (nullable = true)
 |-- Country: string (nullable = true)



In [23]:
ventasTotales.groupBy("Country").count().show()

+-------+-------+
|Country|  count|
+-------+-------+
| France| 314750|
|    USA|1000000|
| Canada|  75318|
+-------+-------+



In [29]:
ventasFinales=ventasTotales.join(productos,ventasTotales.ProductID==productos._c0)

In [30]:
ventasFinales.count()

1390068

In [31]:
ventasFinales.show()

+---------+----------+--------------+-----+---------+-------+----+-------------+-----+------------+---+---------+
|ProductID|      Date|           Zip|Units|  Revenue|Country| _c0|          _c1|  _c2|         _c3|_c4|      _c5|
+---------+----------+--------------+-----+---------+-------+----+-------------+-----+------------+---+---------+
|      726|1999-01-15|75056 CEDEX 01|    1| 115.4500| France| 726| Natura RP-14|Rural|Productivity|  8|   Natura|
|     1909|1999-01-15|75056 CEDEX 01|    2| 398.9000| France|1909| Currus MA-02|  Mix|  All Season|  4|   Currus|
|     1961|1999-01-15|75056 CEDEX 01|    1|  97.0700| France|1961| Currus RP-36|Rural|Productivity|  4|   Currus|
|     1517|1999-01-15|75056 CEDEX 01|    1| 141.6500| France|1517| Quibus RP-09|Rural|Productivity| 12|   Quibus|
|      606|1999-01-15|75056 CEDEX 01|    1| 314.7400| France| 606|Maximus UC-71|Urban| Convenience|  7|VanArsdel|
|     1518|1999-01-15|75056 CEDEX 01|    1| 141.6500| France|1518| Quibus RP-10|Rural|Pr